In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import numpy as np
import pandas as pd
import h5py
from tqdm import tqdm
import scipy.io as sio
from scipy import stats
from importlib import reload
from matplotlib import pyplot as plt
import sys
#sys.path.append('/home/jma819/post_cmfe_analysis')
sys.path.append(r'C:\Users\scanimage\Documents\JJM\post_cnmfe_analysis')
import python_utils_jjm as utils_jjm
import dlc_utils
import statsmodels.formula.api as smf
import scipy.spatial.distance as dist
import itertools
import math
import warnings
import numbers
from multiprocessing import Pool
from functools import partial
warnings.filterwarnings(action='once')

C:\Users\scanimage\AppData\Local\Temp\ipykernel_19208\2024270946.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [ ]:
#import matlab.engine 
#import os

In [ ]:
#eng = matlab.engine.start_matlab()

In [ ]:
#os.chdir('/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/openfield_analysis/spatial_clustering/jones_scripts')
#eng.cd('/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/openfield_analysis/spatial_clustering/jones_scripts')

In [ ]:
## import Jones's functions
#eng.help(eng.getPaddedSignalPeaks())

In [2]:
reload(utils_jjm)
reload(dlc_utils)

<module 'dlc_utils' from 'C:\\Users\\scanimage\\Documents\\JJM\\post_cnmfe_analysis\\dlc_utils.py'>

In [4]:
#cnmfe_file_key = pd.read_csv('/Volumes/My_Passport/cnmfe_analysis_files/OpenFieldAnalysis/2020/cnmfe_file_key.csv')
#cnmfe_file_key = pd.read_csv('/projects/p30771/dlc_analysis/openfield_dlc_output/cnmfe_file_key.csv')

In [22]:
cnmfe_base_dir = r'F:\\JJM\\miniscope_analysis\\mGluR5_NAM\\'
#cnmfe_base_dir = '/projects/p30771/MATLAB/CNMF_E_jjm/quest_MATLAB_cnmfe/batch_output_files/'
vehicle_CNMFE_file = ['24-Aug_18_47_51_out.mat', '24-Aug_18_48_08_out.mat', '26-Aug_20_22_57_out.mat', '30-Aug_12_38_32_out.mat', '31-Aug_12_18_47_out.mat']

fenobam_CNMFE_files = ['25-Aug_17_46_55_out.mat', '26-Aug_15_57_27_out.mat', '30-Aug_18_16_26_out.mat', '31-Aug_13_16_00_out.mat',
                      '31-Aug_13_41_59_out.mat']

In [23]:
#tracking_files = ['/Volumes/My_Passport/dlc_analysis/behavcamvideos/open_field_miniscope/'+utils_jjm.find_behavior_tracking(fname, cnmfe_file_key) for fname in ko_CNMFE_files] 
#tracking_files = ['/projects/p30771/dlc_analysis/openfield_dlc_output/'+utils_jjm.find_behavior_tracking(fname, cnmfe_file_key) for fname in wt_CNMFE_file] 

In [24]:
CNMFE_dir_paths_list = [str(cnmfe_base_dir+fname) for fname in vehicle_CNMFE_file]
#CNMFE_dir_paths_list

In [17]:
cell_fluorescence = sio.loadmat(str(cnmfe_base_dir+'26-Aug_20_22_57_out.mat'))
CNMFE_results = cell_fluorescence

In [18]:
np.shape(cell_fluorescence['C'])

(8, 35682)

In [ ]:
spatial_components = np.array(CNMFE_results['A'].todense())

In [ ]:
spatial_components
dims=(600, 600)
d1 = dims[0]
d2 = dims[1]
coms = utils_jjm.com(spatial_components, d1, d2)

In [ ]:
CNMFE_dir_paths_list[0].split(r'\\')[-1]

In [37]:
grouped_raw_data = {}
success = []
failed = []
#for CNMFE_file, tracking_file in zip(CNMFE_dir_paths_list, tracking_files):
for CNMFE_file in CNMFE_dir_paths_list:
    print(CNMFE_file)
    #print(tracking_file)
    try:
        # load cell fluorescence 
        cell_fluorescence = sio.loadmat(CNMFE_file)
        C_timedelta = utils_jjm.create_fluorescence_time_delta(cell_fluorescence['C'])
        C_normalized = C_timedelta.apply(utils_jjm.normalize).set_index(pd.to_timedelta(np.linspace(0, (len(C_timedelta)-1)*(1/20), len(C_timedelta)), unit='s'), drop=True)
        C_z_scored = C_timedelta.apply(stats.zscore).set_index(pd.to_timedelta(np.linspace(0, (len(C_timedelta)-1)*(1/20), len(C_timedelta)), unit='s'), drop=True)
        C_normalized_z_scored = C_normalized.apply(stats.zscore).set_index(pd.to_timedelta(np.linspace(0, (len(C_normalized)-1)*(1/20), len(C_normalized)), unit='s'), drop=True)
        # create tracking time deltas
        #interpolated = utils_jjm.prepare_timedelta_dfs(tracking_file)
        #load spatial components by session
        #dims of v4 scope are (600,600)
        com_df, spatial_components = utils_jjm.return_spatial_info(CNMFE_file, 0.6, dims=(600, 600))
        cell_contours, for_dims = utils_jjm.create_contour_layouts(spatial_components, dims=(600, 600))
        #C_z_scored_filtered = utils_jjm.filter_out_by_size(C_z_scored, cell_contours, for_dims, 0.6, 100)
        #store results 
        grouped_raw_data[CNMFE_file.split(r'\\')[-1]] = {'C': C_timedelta, 'C_z_scored': C_z_scored, 'C_normalized': C_normalized, 'C_normalized_z_scored': C_normalized_z_scored, 
                                                          'com' : com_df, 'spatial_components' : spatial_components, 'cell_contours': cell_contours,  
                                                         'for_dims' : for_dims}
        success.append(CNMFE_file.split(r'\\')[-1])
    except FileNotFoundError:
        failed.append(tracking_file)
    except OSError:
        failed.append(tracking_file)

F:\\JJM\\miniscope_analysis\\mGluR5_NAM\\24-Aug_18_47_51_out.mat
F:\\JJM\\miniscope_analysis\\mGluR5_NAM\\24-Aug_18_48_08_out.mat
F:\\JJM\\miniscope_analysis\\mGluR5_NAM\\26-Aug_20_22_57_out.mat
F:\\JJM\\miniscope_analysis\\mGluR5_NAM\\30-Aug_12_38_32_out.mat
F:\\JJM\\miniscope_analysis\\mGluR5_NAM\\31-Aug_12_18_47_out.mat


## spatial clustering

In [38]:
#downsample calcium traces, use 'C_normalized_z_scored' data 
new_sampling_interval = .2
downsampled_df = pd.concat([grouped_raw_data[session]['C_normalized_z_scored'].resample(str(new_sampling_interval)+'S').max() 
                  for session in list(grouped_raw_data.keys())], keys=list(grouped_raw_data.keys()))

In [39]:
# compile and filter fluorescence
filtered_for_analysis = {}
for session in list(grouped_raw_data.keys()):
    filtered_for_analysis[session] = utils_jjm.filter_out_by_size(grouped_raw_data[session]['C_normalized_z_scored'], grouped_raw_data[session]['cell_contours'], 
                                                                  grouped_raw_data[session]['for_dims'], 0.6, 100)
C_df = pd.concat([filtered_for_analysis[session].resample(str(new_sampling_interval)+'S').max()
                  for session in list(grouped_raw_data.keys())], keys=list(grouped_raw_data.keys()))


In [45]:
C_df.loc['31-Aug_12_18_47_out.mat'].drop('msCamFrame', axis=1)

,1,2,3,4,5,6,7,8
0 days 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
0 days 00:17:29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:17:29.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:17:29.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:17:29.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# clustering info per session 
coactivity_dfs = {}
for session in tqdm(list(grouped_raw_data.keys())):
    # get indicies of small cells 
    cells_to_drop = np.array([cell for cell in range(1, len(grouped_raw_data[session]['cell_contours'])+1) if 
                              len(np.array(np.where(grouped_raw_data[session]['for_dims'][cell]>0.6)[0]))<100])
    #compile and filter cell centers of mass
    filtered_centers_of_mass = grouped_raw_data[session]['com'].drop(cells_to_drop, axis =0)
    coactivity_dfs[session] = filtered_centers_of_mass

100%|████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<?, ?it/s]


In [42]:
com_df = pd.concat(coactivity_dfs)

In [43]:
## save outputs if needed 
#com_df.loc['GRIN026_H16_M35_S34'].to_csv('/Volumes/My_Passport/cnmfe_analysis_files/OpenFieldAnalysis/2020/GRIN026_H16_M35_S34_com.csv')
#com_df.to_csv('/Volumes/My_Passport/cnmfe_analysis_files/OpenFieldAnalysis/2020/wt_com.csv')
save_path=r'F:\\JJM\\miniscope_analysis\\mGluR5_NAM\\'
for session in list(grouped_raw_data.keys()):
    com_df.loc[session].to_csv(save_path+session+'_com.csv')
    C_df.loc[session].drop('msCamFrame', axis=1).to_csv(save_path+session+'_filtered_f_traces.csv')
#com_df.to_csv('/projects/p30771/miniscope/analysis/OpenFieldAnalysis/spatial_clusters/jones_script_analysis/wt_com.csv')

KeyError: '31-Aug_12_18_47_out.mat'

In [ ]:
list(grouped_raw_data.keys())

In [ ]:
C_df.loc['GRIN012_H17_M32_S17']

In [ ]:
## start analysis with Jones's scripts

In [ ]:
C_df.loc['GRIN026_H16_M35_S34'].iloc[:,:].dropna(axis=1).values.transpose()[:-1,:]

In [ ]:
#get fluorescence traces for one animal
cell_traces = C_df.loc['GRIN026_H16_M35_S34'].iloc[:,:].dropna(axis=1).values.transpose()[:-1,:]

In [ ]:
cell_traces

In [ ]:
np.shape(cell_traces)[0]

In [ ]:
#xy centroids
cell_XY_coords = com_df.loc['GRIN026_H16_M35_S34'].iloc[:,:].dropna(axis=1).values.transpose()

In [ ]:
#convert to MATLAB array 
cell_traces_MATLAB = matlab.double(cell_traces.tolist())
cell_XY_coords_MATLAB = matlab.double(cell_XY_coords.tolist())

In [ ]:
#output signal peaks 
signal_peaks = eng.computeSignalPeaks(cell_traces_MATLAB, 'doMovAvg', 0, 'reportMidpoint', 1, 'numStdsForThresh', 2.5)

In [ ]:
#pad signal peaks
padded_signal_peaks = eng.getPaddedSignalPeaks(signal_peaks)

In [ ]:
#eng.size(signal_peaks)
eng.size(padded_signal_peaks)

In [ ]:
#get cell distances multiply by pixel to micron scale
cellDistances = eng.pdist(cell_XY_coords_MATLAB, 'euclidean')*2.5

In [ ]:
#get velocity bins
#reset timedelta index to integers matching frame numbers
vbin_frames_by_session = {}
for session in list(grouped_raw_data.keys()):
    df_w_integer_index = C_df.loc[session].reset_index()
    frame_indicies = {}
    for interval_range in all_sessions_v_bins.values.categories.values:
        frame_indicies[interval_range] = df_w_integer_index[df_w_integer_index['velocity_bins']==interval_range].index.values
    vbin_frames_by_session[session] = frame_indicies

In [ ]:
#pull out indicies matching given bins 
#save list to csv file

for session in list(grouped_raw_data.keys()):
    for interval_range_int in range(len(all_sessions_v_bins.values.categories.values)):
        np.savetxt('/projects/p30771/miniscope/analysis/OpenFieldAnalysis/spatial_clusters/jones_script_analysis/'+str(session)+'_frame_bins_'+str(interval_range_int).zfill(3)+'_.csv',
                vbin_frames_by_session[session][list(frame_indicies.keys())[interval_range_int]])
    

In [ ]:
#input frames to analyze
# all frames
frames_to_analyze = matlab.double(np.linspace(0, np.shape(cell_traces)[1]-1, np.shape(cell_traces)[1]).tolist())
within_frames_to_analyze = matlab.double(np.linspace(0, np.shape(cell_traces)[1]-1, np.shape(cell_traces)[1]).tolist())

In [ ]:
# frames subset
v_bin_to_analyze = 0 

frames_to_analyze = matlab.double(frame_indicies[list(frame_indicies.keys())[v_bin_to_analyze]].tolist())
within_frames_to_analyze = matlab.double(frame_indicies[list(frame_indicies.keys())[v_bin_to_analyze]].tolist())

In [ ]:
# cell distance bins
maxDist = matlab.double([500]) #compare out to 500 um
binSize = matlab.double([20]) #compare in 20 um bins
numBins = matlab.double([maxDist[0][0]/binSize[0][0]-1]) #throw out the first bin (overlapping cells)
binVector = matlab.double(np.linspace(int(binSize[0][0]), int(maxDist[0][0]), int(numBins[0][0])+1).tolist())

In [ ]:
#other needed variables
numCells = matlab.double([np.shape(cell_traces)[0]])

In [ ]:
# get Jaccard scores and compare to shuffle

eng.p355_jaccard_shuffle(padded_signal_peaks, frames_to_analyze[0], within_frames_to_analyze, cell_XY_coords_MATLAB, numCells, cellDistances, numBins, binVector, 1)


In [ ]:
# clustering info per session 
coactivity_dfs = {}
for session in tqdm(list(grouped_raw_data.keys())):
    # get indicies of small cells 
    cells_to_drop = np.array([cell for cell in range(1, len(grouped_raw_data[session]['cell_contours'])+1) if 
                              len(np.array(np.where(grouped_raw_data[session]['for_dims'][cell]>0.6)[0]))<100])
    #compile and filter cell centers of mass
    filtered_centers_of_mass = grouped_raw_data[session]['com'].drop(cells_to_drop, axis =0)

    com_distances = utils_jjm.get_pairwise_distance_by_session(filtered_centers_of_mass)

    ##get binned fluorescence and calc Jaccard scores
    #arguments are sample widths to bin and z score threshold
    cells_in_session = C_df.loc[session].dropna(axis=1).drop('msCamFrame', axis=1)
    binned_fluorescence = cells_in_session.apply(utils_jjm.binning_function_uncrop, args=[1, 2])

    reindexed = binned_fluorescence.set_index(int(x) for x in np.linspace(0, len(binned_fluorescence)-1, len(binned_fluorescence)))

    #ks_results_2sided, ks_one_sided_more, ks_one_sided_less, coactivity_dfs = utils_jjm.spatial_coordination_by_session(reindexed, 10, com_distances)
    coactivity_dfs[session] = utils_jjm.spatial_coordination_by_session(reindexed)
    
    

In [ ]:
#save coactivity info to h5py file
for session in tqdm(list(grouped_raw_data.keys())):
    f = h5py.File("/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/openfield_analysis/spatial_clustering/" + str(session) +"clustering.h5", "w")

    for time_pt in range(len(coactivity_dfs[session])):
        dset = f.create_dataset(str(time_pt), np.shape(coactivity_dfs[session][time_pt].values), dtype=np.float32)
        dset[:, :] = coactivity_dfs[session][time_pt].values

    cell_pair_ids = f.create_dataset('cell_ids', np.shape(list(coactivity_dfs[session][0].columns)), dtype='i')
    cell_pair_ids[:, :] = list(coactivity_dfs[session][0].columns)

    f.close()

In [ ]:
spatial_coordination_by_session = {}
for session in list(grouped_raw_data.keys()):
    print(session)
    f = h5py.File("/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/openfield_analysis/spatial_clustering/" + str(session) +"clustering.h5", "r")
# get indicies of small cells 
    cells_to_drop = np.array([cell for cell in range(1, len(grouped_raw_data[session]['cell_contours'])+1) if 
                          len(np.array(np.where(grouped_raw_data[session]['for_dims'][cell]>0.6)[0]))<100])
#compile and filter cell centers of mass
    filtered_centers_of_mass = grouped_raw_data[session]['com'].drop(cells_to_drop, axis =0)
    com_distances = utils_jjm.get_pairwise_distance_by_session(filtered_centers_of_mass)

# get one sided ks tests from h5 data set
    binnums = 10
    ks_one_sided_more = []
    ks_one_sided_less = []
    for time_point in time_points:
        active_cells = [cell_ids[x] for x in [index[0] for index in np.argwhere(f[str(time_points[time_point])][0, 0:len(cell_ids)])]]
        active_cell_distances = [com_distances[cell].values[0] for cell in active_cells]
        cum_results_coactive = stats.cumfreq(active_cell_distances, numbins=binnums, defaultreallimits=(0, 500))
        less_result = stats.kstest(cum_results_coactive.cumcount/len(active_cell_distances), 'norm', alternative='less')
        more_result = stats.kstest(cum_results_coactive.cumcount/len(active_cell_distances), 'norm', alternative='greater')
        ks_one_sided_more.append(more_result)
        ks_one_sided_less.append(less_result)
    
    spatial_coordination_by_session[session] = {'ks_one_sided_more' : ks_one_sided_more, 'ks_one_sided_less' : ks_one_sided_less}

In [ ]:
session = list(grouped_raw_data.keys())[2]

In [ ]:
f = h5py.File("/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/openfield_analysis/spatial_clustering/" + str(session) +"clustering.h5", "r")
# get indicies of small cells 
cells_to_drop = np.array([cell for cell in range(1, len(grouped_raw_data[session]['cell_contours'])+1) if 
                          len(np.array(np.where(grouped_raw_data[session]['for_dims'][cell]>0.6)[0]))<100])
#compile and filter cell centers of mass
filtered_centers_of_mass = grouped_raw_data[session]['com'].drop(cells_to_drop, axis =0)
com_distances = utils_jjm.get_pairwise_distance_by_session(filtered_centers_of_mass)

In [ ]:
cell_ids = f['cell_ids']

In [ ]:
time_point = 95

In [ ]:
len(cell_ids)

In [ ]:
cell_ids[10]

In [ ]:
np.argwhere(f[str(time_point)][0, 0:len(cell_ids)])

In [ ]:
active_cells = [cell_idx[0] for cell_idx in np.argwhere(f[str(time_point)][0, 0:len(cell_ids)])]

In [ ]:
len(filtered_centers_of_mass)

In [ ]:
[cell_ids[cell] for cell in active_cells]

In [ ]:
cell_ids[active_cells[0]][0]

In [ ]:
cell_ids[active_cells[0]][1]

In [ ]:
com_distances[5]

In [ ]:
[com_distances[cell_ids[cell_idx][0]][cell_ids[cell_idx][1]].values[0] for cell_idx in active_cells]

In [ ]:
for time_pt in range(6251):
    if len(np.argwhere(f[str(time_pt)][0, 0:len(cell_ids)])) > 1:
        print(time_pt)
        break 
    

In [ ]:
len(f.keys())

In [ ]:
# get one sided ks tests from h5 data set
binnums = 10
ks_one_sided_more = []
ks_one_sided_less = []
cell_ids = f['cell_ids']
for time_point in time_points:
    print(time_point)
    active_cells = [cell_idx[0] for cell_idx in np.argwhere(f[str(time_point)][0, 0:len(cell_ids)])]
    active_cell_distances = [com_distances[cell_ids[cell_idx][0]][cell_ids[cell_idx][1]].values[0] for cell_idx in active_cells]
    cum_results_coactive = stats.cumfreq(active_cell_distances, numbins=binnums, defaultreallimits=(0, 500))
    less_result = stats.kstest(cum_results_coactive.cumcount/len(active_cell_distances), 'norm', alternative='less')
    more_result = stats.kstest(cum_results_coactive.cumcount/len(active_cell_distances), 'norm', alternative='greater')
    ks_one_sided_more.append(more_result)
    ks_one_sided_less.append(less_result)
    
spatial_coordination_by_session[session] = {'ks_one_sided_more' : ks_one_sided_more, 'ks_one_sided_less' : ks_one_sided_less}

In [ ]:
coord_index = np.nan_to_num([math.log(result.pvalue, 10)*-1 for result in ks_one_sided_less])

In [ ]:
plt.plot(coord_index[1700:2500])
plt.show()

In [ ]:
plt.plot(V_df.loc[session]['tail_base'].values[1700:2500])
plt.show()

In [ ]:
## for plotting average spatial coordination during trigger regions

In [ ]:

triggered_activity_across_sessions = {}
for session in list(grouped_raw_data.keys()):
    #inputs activity_threshold, resting_threshold, resting_baseline 
    crossing_indicies = utils_jjm.select_trigger_regions(binned_velocity_df[session], 0.5, 0.5, 20)
    #inputs time_to_plot
    threshold_activity = utils_jjm.average_triggered_regions(V_df.loc[session]['tail_base'].values, crossing_indicies, 40)
    
    f_threshold_activity = utils_jjm.average_triggered_regions(spatial_coordination_activity_in_session 
            , crossing_indicies, 40)
    triggered_activity_across_sessions[session] = pd.concat([threshold_activity, f_threshold_activity], axis=1, keys=['velocity', 'spatial_coordination'])

In [ ]:
#triggered_activity_across_sessions

In [ ]:
#concat by mouse 
concacted_by_mouse = {}
for mouse in list(set([session[0:7] for session in list(triggered_activity_across_sessions.keys())])):
    dfs_by_mouse = []
    sessions = []
    for session in list(triggered_activity_across_sessions.keys()):
        if mouse in session:
            if not(triggered_activity_across_sessions[session].empty):
                means = triggered_activity_across_sessions[session].mean(axis=1, level=0)
                dfs_by_mouse.append(means)
                sessions.append(session)
    if len(dfs_by_mouse)>0:
        concacted_by_mouse[mouse] = pd.concat(dfs_by_mouse, axis=1, keys=sessions)
combined_by_mouse = pd.concat(list(concacted_by_mouse.values()), axis=1, keys=list(concacted_by_mouse.keys()))

combined_by_mouse.head()

In [ ]:
binnums = 10
#ks_results_2sided = []
ks_one_sided_more = []
ks_one_sided_less = []
for time_point in tqdm(time_points):
    active_cells = [cell_ids[x] for x in [index[0] for index in np.argwhere(f[str(time_points[time_point])][0, 0:len(cell_ids)])]]
    coactive_cell_distances = [com_distances[cell].values[0] for cell in active_cells]
    #non_coactive_distances = com_distances[coactivity_df[coactivity_df == 0].dropna(axis=1).columns]
    #linear_distribution = np.linspace(1, len(coactive_cell_distances.columns), binnums)
    #binnums=10
    cum_results_coactive = stats.cumfreq(coactive_cell_distances, numbins=binnums, defaultreallimits=(0, 500))
    #cum_results_non_coactive = stats.cumfreq(non_coactive_distances.values[0], numbins=binnums, defaultreallimits=(0, 500))
    #plt.plot(np.linspace(0, 500, binnums), cum_results_coactive.cumcount/len(coactive_cell_distances.values[0]))
    #plt.plot(np.linspace(0, 500, binnums), cum_results_non_coactive.cumcount/len(non_coactive_distances.values[0]))
    #plt.plot(np.linspace(0, 500, binnums), linear_distribution/len(coactive_cell_distances.values[0]))
    #plt.show()
    less_result = stats.kstest(cum_results_coactive.cumcount/len(coactive_cell_distances), 'norm', alternative='less')
    more_result = stats.kstest(cum_results_coactive.cumcount/len(coactive_cell_distances), 'norm', alternative='greater')
    ks_one_sided_more.append(more_result)
    ks_one_sided_less.append(less_result)
    #coordination_index = utils_jjm.create_coordination_index(more_result, less_result)
    #two sided test between coactive and noncoactive distribution 
    #ks_result = stats.ks_2samp(cum_results_coactive.cumcount/len(coactive_cell_distances.values[0]), cum_results_non_coactive.cumcount/len(non_coactive_distances.values[0]))
    #coordination_indicies.append(coordination_index)
    #ks_results_2sided.append(ks_result)
    

In [ ]:
x = np.linspace(-40*.2, 40*.2, 80)
mean=combined_by_mouse.mean(axis=1, level=2)['velocity'].values
plt.plot(x, combined_by_mouse.mean(axis=1, level=2)['velocity'].values, color='k')
std_error = (combined_by_mouse.std(axis=1, level=2)['velocity'])/math.sqrt(combined_by_mouse.mean(axis=1, level=0).shape[1])
plt.fill_between(x, mean-std_error, mean+std_error)
ax = plt.gca()
ax.axvline(x=(0), linestyle='--', color='r')
plt.show()

In [ ]:
x = np.linspace(-40*.2, 40*.2, 80)
mean = combined_by_mouse.mean(axis=1, level=2)['fluorescence'].values
plt.plot(x, mean, color='k')
std_error = (combined_by_mouse.std(axis=1, level=2)['fluorescence'])/math.sqrt(combined_by_mouse.mean(axis=1, level=0).shape[1])
plt.fill_between(x, mean-std_error, mean+std_error)
ax = plt.gca()
ax.axvline(x=0, linestyle='--', color='r')
plt.show()